In [651]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pandas.io import data, wb
import datetime
import os.path
import math


In [652]:
symbol = 'SPY'
df_train = pd.read_csv('data/'+symbol+'-ohlc.csv', index_col='Date', parse_dates=True)
if (os.path.isfile('data/'+symbol+'-dividends.csv')):
    df_dividends = pd.read_csv('data/'+symbol+'-dividends.csv', index_col='Date', parse_dates=True)
    df_train = df_train.join(df_dividends)
    df_train['dividend'].fillna(0, inplace=True)
else:
    df_train['dividend']=0

In [653]:
#df_train['Close'].pct_change().hist(bins=100)
#plt.show()

In [654]:
pma10 = df_train['Close'].rolling(window=10,center=False).mean() + df_train['dividend'].rolling(window=10, center=False).mean()
pma20 = df_train['Close'].rolling(window=20,center=False).mean()+ df_train['dividend'].rolling(window=20, center=False).mean()
pma50 = df_train['Close'].rolling(window=50,center=False).mean()+ df_train['dividend'].rolling(window=50, center=False).mean()
pma200 = df_train['Close'].rolling(window=200,center=False).mean()+ df_train['dividend'].rolling(window=200, center=False).mean()
vma50 = df_train['Volume'].rolling(window=50, center=False).mean()



In [655]:
#distance from MAs
diffma10  = (df_train['Close']/pma10 -1)*100
diffma20 = (df_train['Close']/pma20 -1)*100
diffma50 = (df_train['Close']/pma50 -1)*100
diffma200 = (df_train['Close']/pma200 -1)*100

#tangential of ma lines
pma10tan = (pma10[1:]/pma10[:-1].values-1)*100
pma20tan = (pma20[1:]/pma20[:-1].values-1)*100
pma50tan = (pma50[1:]/pma50[:-1].values-1)*100
pma200tan = (pma200[1:]/pma200[:-1].values-1)*100

#sequence of higher high/higher low
hhx1 = df_train['High'][1:] + df_train['dividend'][1:] > df_train['High'][:-1] 
hlx1 = df_train['Low'][1:] + df_train['dividend'][1:] > df_train['Low'][:-1] 
hhx2 = hhx1.shift(1)
hhx3 = hhx1.shift(2)
hlx2 = hlx1.shift(1)
hlx3 = hlx1.shift(2)

#rolling average of hh and hl
hh10 = hhx1.rolling(window=10, center=False).mean()
hl10 = hlx1.rolling(window=10, center=False).mean()

nvx1 = df_train['Volume']/vma50

rhigh = (df_train['High']/df_train['Close'] - 1)*100
rlow = (df_train['Low']/df_train['Close'] - 1)*100
ropen = (df_train['Open'][1:]/df_train['Close'][:-1].values-1)*100
#shift up to make it align
#ropen = ropen.shift(1)
rbody = ((df_train['Open'] - df_train['Close'])/df_train['Close'])*100

In [656]:
#print(len(ropen))
#print(df_train['Open'][1:].head(), df_train['Close'][:-1].head(), ropen.head())

In [657]:
#print(ropen.tail())


In [658]:
#add features for training data

#distance from MAs
df_train = df_train.assign(diffma10=diffma10)
df_train = df_train.assign(diffma20=diffma20)
df_train = df_train.assign(diffma50=diffma50)
df_train = df_train.assign(diffma200=diffma200)

#tangential of ma lines
#df_train = df_train.assign(pma10tan=pma10tan)
#df_train = df_train.assign(pma20tan=pma20tan)
#df_train = df_train.assign(pma50tan=pma50tan)
#df_train = df_train.assign(pma200tan=pma200tan)

df_train = df_train.assign(hhx1=hhx1)
df_train = df_train.assign(hhx2=hhx2)
df_train = df_train.assign(hhx3=hhx3)

df_train = df_train.assign(hlx1=hlx1)
df_train = df_train.assign(hlx2=hlx2)
df_train = df_train.assign(hlx3=hlx3)

df_train = df_train.assign(hh10=hh10)
df_train = df_train.assign(hl10=hl10)

#last day volume
df_train = df_train.assign(nvx1=nvx1)

#candle sticks
df_train = df_train.assign(rhigh=rhigh)
df_train = df_train.assign(rlow=rlow)
df_train = df_train.assign(ropen=ropen)
df_train = df_train.assign(rbody=rbody)

In [659]:
df_train.to_csv('data/'+symbol+'-train.csv')
remove = ['Open','High','Low','Close','Volume','Adj Close','dividend']
df_train = df_train.drop(remove, axis=1)

In [660]:
df_train.describe()


,diffma10,diffma20,diffma50,diffma200,hh10,hl10,nvx1,rhigh,rlow,ropen,rbody
count,5826.000000,5816.000000,5786.000000,5636.000000,5825.000000,5825.000000,5786.000000,5835.000000,5835.000000,5834.000000,5835.000000
mean,0.116091,0.256608,0.662828,2.818657,0.536103,0.541030,1.039993,0.649055,-0.708989,0.032345,0.008871
std,1.775464,2.505600,3.929774,8.414745,0.162091,0.158631,0.529397,0.788404,0.761130,0.643007,1.004828
min,-15.974173,-21.374952,-27.180044,-39.537778,0.000000,0.000000,0.015866,0.000000,-11.248151,-8.321522,-8.505539
25%,-0.752062,-0.948909,-1.172832,-1.027703,0.400000,0.400000,0.739402,0.154909,-0.926656,-0.220981,-0.478986
50%,0.302169,0.577577,1.141036,4.101693,0.500000,0.500000,0.943974,0.394050,-0.500634,0.052130,-0.042595
75%,1.144583,1.763332,3.133244,8.489963,0.600000,0.600000,1.224088,0.851541,-0.252379,0.308369,0.458161
max,8.571041,10.362980,14.114149,20.452184,1.000000,1.000000,12.385631,10.937162,0.000000,6.067800,9.878729


In [661]:
df_train.corr()

,diffma10,diffma20,diffma50,diffma200,hh10,hl10,nvx1,rhigh,rlow,ropen,rbody
diffma10,1.000000,0.875730,0.611042,0.325509,0.522364,0.527854,-0.328661,-0.564335,-0.082418,0.255960,-0.460537
diffma20,0.875730,1.000000,0.813987,0.458150,0.632236,0.628910,-0.352273,-0.531301,0.057314,0.182881,-0.337530
diffma50,0.611042,0.813987,1.000000,0.693141,0.525588,0.492371,-0.288863,-0.489903,0.209221,0.125856,-0.209621
diffma200,0.325509,0.458150,0.693141,1.000000,0.370818,0.333388,-0.093637,-0.407803,0.294235,0.093255,-0.088927
hh10,0.522364,0.632236,0.525588,0.370818,1.000000,0.672927,-0.228775,-0.221753,0.140387,0.068639,-0.076257
hl10,0.527854,0.628910,0.492371,0.333388,0.672927,1.000000,-0.268130,-0.251605,0.168743,0.066654,-0.083602
nvx1,-0.328661,-0.352273,-0.288863,-0.093637,-0.228775,-0.268130,1.000000,0.259387,-0.209776,-0.098766,0.076881
rhigh,-0.564335,-0.531301,-0.489903,-0.407803,-0.221753,-0.251605,0.259387,1.000000,0.155151,-0.152215,0.727703
rlow,-0.082418,0.057314,0.209221,0.294235,0.140387,0.168743,-0.209776,0.155151,1.000000,-0.029322,0.639725
ropen,0.255960,0.182881,0.125856,0.093255,0.068639,0.066654,-0.098766,-0.152215,-0.029322,1.000000,-0.000239


In [662]:
def train(X_train, y_train):  
    from sklearn.cross_validation import train_test_split
    from sklearn import cross_validation
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC


    model = RandomForestClassifier(n_estimators=1000)     #best so far
    #model = AdaBoostClassifier(n_estimators=100)
    #model = LogisticRegression(random_state=1)
    #model = GradientBoostingClassifier(n_estimators=100)
    #model = KNeighborsClassifier(n_neighbors=15)
    
    
    from sklearn import metrics
    
    #y_eval_comp = y_eval_pred[:,1] > 0.55
    #numpy.count_nonzero(y_eval_comp)
    
    #print("recall:", metrics.recall_score(y_eval, y_eval_comp, average='binary'))
    #print("precision:", metrics.precision_score(y_eval, y_eval_comp, average='binary'))
    #print("f1 score:", metrics.f1_score(y_eval, y_eval_comp, average='binary'))
    
    scores = cross_validation.cross_val_score(model, X_train, y_train, cv=5, scoring='precision')
    print("Precision score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
    
    model.fit(X_train, y_train)
    return model



In [ ]:
X_train = df_train.values[200:-10, :]
X_test = df_train.values[-10:,:]
print(len(X_train))



5625


In [ ]:
from sklearn.externals import joblib

#for hh, shift one position back to get future value for hh
print("predict hh:(vs actual)")
y_train = df_train['hhx1'].shift(-1).values[200:-10].astype(bool)

f_name = 'models/predict-hh-'+symbol+'/model.pkl'
if (not os.path.isfile(f_name)):
    model = train(X_train, y_train)
    joblib.dump(model, f_name) 
else:
    model = joblib.load(f_name) 
    
y_test_pred = model.predict_proba(X_test);

print(df_train['hhx1'].values[-9:].astype(bool))
print(y_test_pred[:, 1]>0.5)
print("prob of higher high ===", y_test_pred[-1,1])

#for hl
print("\npredict hl:(vs actual)")
y_train = df_train['hlx1'].shift(-1).values[200:-10].astype(bool)

f_name = 'models/predict-hl-'+symbol+'/model.pkl'
if (not os.path.isfile(f_name)):
    model = train(X_train, y_train)
    joblib.dump(model, f_name) 
else:
    model = joblib.load(f_name) 
    
y_test_pred = model.predict_proba(X_test);

print(df_train['hlx1'].values[-9:].astype(bool))
print(y_test_pred[:, 1]>0.5)
print("prob of higher low ===", y_test_pred[-1,1])



predict hh:(vs actual)
Precision score: 0.69 (+/- 0.02)


In [ ]:
def e_predict(X_train, y_train, X_test):
    from sklearn import cross_validation
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.ensemble import VotingClassifier

    clf1 = LogisticRegression(random_state=1)
    clf2 = RandomForestClassifier(n_estimators=350)
    clf3 = GaussianNB()
    clf4 = GradientBoostingClassifier(n_estimators=100)
    clf5 = KNeighborsClassifier(n_neighbors=15)
    
    eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3), 
                                        ('gb', clf4), ('knn',clf5)], voting='soft')

    for clf in [clf1, clf2, clf3, clf4, clf5, eclf]: 
        print(type(clf).__name__, "\n")
        scores = cross_validation.cross_val_score(clf, X_train, y_train, cv=5, scoring='precision')
        print("Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
        clf.fit(X_train, y_train)
        y_test_pred = clf.predict_proba(X_test);
        return y_test_pred
